In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

#Plotly imports
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'

#From https://github.com/ee-davies/sc-data-functions
import sys
sys.path.insert(1, '/Users/emmadavies/Documents/sc-data-functions')
import functions_solo as solo
import functions_noaa as noaa
import functions_sta as sta

import data_frame_transforms as transform
import functions_insitu_analysis as insitu_tools

# Create pickle files for SolO, DSCOVR, STEREO-A

In [2]:
#input start and end time of data received
start_timestamp = datetime(2024, 3, 1)
# end_timestamp = datetime(2024, 3, 23)

In [3]:
solo.create_solo_pkl(start_timestamp, level='ll')

ERROR: NO_SUCH_CDF: The specified CDF does not exist. /Volumes/External/data/solo/mag/ll/solo_L2_mag-rtn-ll-internal_20240305_V00.cdf
ERROR: NO_SUCH_CDF: The specified CDF does not exist. /Volumes/External/data/solo/mag/ll/solo_L2_mag-rtn-ll-internal_20240306_V00.cdf
ERROR: NO_SUCH_CDF: The specified CDF does not exist. /Volumes/External/data/solo/swa/plas/l2/solo_L2_swa-pas-grnd-mom_20240301.cdf
ERROR: NO_SUCH_CDF: The specified CDF does not exist. /Volumes/External/data/solo/swa/plas/l2/solo_L2_swa-pas-grnd-mom_20240302.cdf
ERROR: NO_SUCH_CDF: The specified CDF does not exist. /Volumes/External/data/solo/swa/plas/l2/solo_L2_swa-pas-grnd-mom_20240303.cdf
ERROR: NO_SUCH_CDF: The specified CDF does not exist. /Volumes/External/data/solo/swa/plas/l2/solo_L2_swa-pas-grnd-mom_20240304.cdf
ERROR: NO_SUCH_CDF: The specified CDF does not exist. /Volumes/External/data/solo/swa/plas/l2/solo_L2_swa-pas-grnd-mom_20240305.cdf
ERROR: NO_SUCH_CDF: The specified CDF does not exist. /Volumes/External/

In [4]:
noaa.create_dscovr_pkl()

# Load in pickle files

In [5]:
# create str times for prediction dating/file use
t_now = datetime.utcnow()
t_now_date = datetime.utcnow().strftime("%Y-%m-%d")
t_now_date_hour = datetime.utcnow().strftime("%Y-%m-%d-%H")

In [6]:
# unpickle dataframes
output_path='/Users/emmadavies/Documents/Projects/SolO_Realtime_Preparation/March2024/'

obj_solo = pd.read_pickle(output_path+f'solo_rtn_{t_now_date}.p')
solo_df = pd.DataFrame.from_records(obj_solo[0])

obj_dscovr = pd.read_pickle(output_path+f'dscovr_gsm_{t_now_date_hour}.p')
dscovr_df = pd.DataFrame.from_records(obj_dscovr[0])

# Plot Solar Orbiter MAG Data (Original RTN)

In [7]:
#define solo info for plot titles
t_start = solo_df['time'][0]
t_end = solo_df['time'][solo_df.shape[0]-1]

lag = (t_now-t_end)/timedelta(hours=1)

r_recent = solo_df['r'][solo_df.shape[0]-1]
lat_recent = solo_df['lat'][solo_df.shape[0]-1]
lon_recent = solo_df['lon'][solo_df.shape[0]-1]

In [8]:
nrows = 1
fig = make_subplots(rows=nrows, cols=1, shared_xaxes=True)

for column, color in zip(['bx', 'by', 'bz', 'bt'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=solo_df['time'],
            y=solo_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=1, col=1
    )

fig.update_yaxes(title_text='B [nT]', row=1, col=1)

fig.update_yaxes(showgrid=True, zeroline=False, showticklabels=True,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)
fig.update_xaxes(showgrid=True, zeroline=False, showticklabels=True, rangeslider_visible=False,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)

fig.update_layout(
    title=f'SolO MAG Data in RTN from {t_start} to {t_end} <br>Latest SolO Position: r = {r_recent:.2f} AU, lat = {lat_recent:.2f}, lon = {lon_recent:.2f} <br>Time lag to current time: {lag:.2f} hours')

#fig.write_html(f'{output_path}/SolO_MAG_{t_now_date}.html')

fig.show()

## Define SolO boundary times and give ELEvo outputs:

In [9]:
t_shock = datetime(2024, 3, 1, 13, 10)
t_le = datetime(2024, 3, 1, 19, 2)
t_te = datetime(2024, 3, 2, 4, 26)

#from ELEvo
speed = 424.21
donki_id = "2024-02-27T11:36:00-CME-001"
L1_pred_arrival_time = datetime(2024, 3, 3, 8, 48)
L1_pred_arrival_uncertainty = 0.5 #given in hours, always even +/- uncertainty

## Plot SolO in RTN with boundaries and info: share this plot!

In [10]:
shock_colour = "#BD88E8"
le_colour = "#FF8300"
te_colour = "#00D4FF"

nrows = 1
fig = make_subplots(rows=nrows, cols=1, shared_xaxes=True)

for column, color in zip(['bx', 'by', 'bz', 'bt'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=solo_df['time'],
            y=solo_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=1, col=1
    )

fig.update_yaxes(title_text='B [nT] RTN', row=1, col=1)

fig.add_vline(x=t_shock, row=1, col=1, line_width=1, line_dash="dash", line_color=shock_colour)
fig.add_vline(x=t_le, row=1, col=1, line_width=1, line_dash="dash", line_color=le_colour)
fig.add_vline(x=t_te, row=1, col=1, line_width=1, line_dash="dash", line_color=te_colour)

fig.update_yaxes(showgrid=True, zeroline=False, showticklabels=True,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)
fig.update_xaxes(showgrid=True, zeroline=False, showticklabels=True, rangeslider_visible=False,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)

fig.update_layout(
    title=f'SolO MAG Data in RTN from {t_start} to {t_end} <br>Latest SolO Position (HEEQ): r = {r_recent:.2f} AU, lat = {lat_recent:.2f}, lon = {lon_recent:.2f} <br>Time lag to current time: {lag:.2f} hours <br>From ELEvo: Estimated arrival speed at SolO: {speed} km/s, DONKI ID: {donki_id} <br>ICME start: {t_shock}, MO start: {t_le}, MO end: {t_te}',
    margin={"t": 200})

fig.write_html(f'{output_path}/SolO_MAG_{t_now_date}.html')

fig.show()

# Time-Shifting, ICME Expansion, Magnetic Field Scaling, Coord Conversion for SolO

In [11]:
#Transform SolO MAG data from RTN to GSM coordinates
df_solo_gsm = transform.RTN_to_GSM(solo_df)

In [12]:
#If using predicted ELEvo arrival speed to timeshift:
# df_timeshifted = insitu_tools.timeshift_dataframe_predspeed(df_solo_gsm, speed)

#If using predicted ELEvo L1 arrival time to timeshift (preferred):
df_timeshifted = insitu_tools.timeshift_dataframe_predtime(df_solo_gsm, t_shock, L1_pred_arrival_time)

In [13]:
#Expand timeshifted ICME (currently set to expand between MO start and end, but could change to ICME start)
df_expanded = insitu_tools.expand_icme(df_timeshifted, dscovr_df, t_shock, t_te)

/Users/emmadavies/Documents/sc-data-functions/functions_insitu_analysis.py:46: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/Users/emmadavies/Documents/sc-data-functions/functions_insitu_analysis.py:49: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.



In [14]:
#If using predicted ELEvo arrival speed to timeshift:
# ts_s, ts_s_lb, ts_s_ub = insitu_tools.timeshift_boundary_predspeed(t_shock, solo_df, speed)
# ts_le, ts_le_lb, ts_le_ub = insitu_tools.timeshift_boundary_predspeed(t_le, solo_df, speed)
# ts_te, ts_te_lb, ts_te_ub = insitu_tools.timeshift_boundary_predspeed(t_te, solo_df, speed)

#If using predicted ELEvo L1 arrival time to timeshift (preferred):
ts_s, ts_s_lb, ts_s_ub = insitu_tools.timeshift_boundary_predtime(df_expanded, t_shock, L1_pred_arrival_uncertainty)
ts_le, ts_le_lb, ts_le_ub = insitu_tools.timeshift_boundary_predtime(df_expanded, t_le, L1_pred_arrival_uncertainty)
ts_te, ts_te_lb, ts_te_ub = insitu_tools.timeshift_boundary_predtime(df_expanded, t_te, L1_pred_arrival_uncertainty)

/Users/emmadavies/Documents/sc-data-functions/functions_insitu_analysis.py:91: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/Users/emmadavies/Documents/sc-data-functions/functions_insitu_analysis.py:95: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/Users/emmadavies/Documents/sc-data-functions/functions_insitu_analysis.py:99: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/Users/emmadavies/Documents/sc-data-functions/functions_insitu_analysis.py:91: FutureWarning:

Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/Users/emmadavies/Documents/sc-data-functions/functi

In [15]:
#scale B field data using power law (default = -1.6)
df_scaled = insitu_tools.scale_B_field(df_expanded, dscovr_df)

# Final Prediction:
## Plot of Time-Shifted, Expanded, and Scaled B of SolO, incl. uncertainties
## Includes DSCOVR real-time data for comparison

In [26]:
shock_colour = "#BD88E8"
le_colour = "#FF8300"
te_colour = "#00D4FF"


nrows = 5
fig = make_subplots(rows=nrows, cols=1, shared_xaxes=True, 
                    specs=[[{"secondary_y": False}],
                           [{"secondary_y": False}],
                           [{"secondary_y": False}],
                           [{"secondary_y": False}],
                           [{"secondary_y": True}]])

for column, color in zip(['bx', 'by', 'bz', 'bt'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=df_solo_gsm['time'],
            y=df_solo_gsm[column],
            name=column.upper(),
            line_color=color
        ),
        row=1, col=1
    )

fig.add_vline(x=t_shock, row=1, col=1, line_width=1.5, line_dash="dash", line_color=shock_colour)
fig.add_vline(x=t_le, row=1, col=1, line_width=1.5, line_dash="dash", line_color=le_colour)
fig.add_vline(x=t_te, row=1, col=1, line_width=1.5, line_dash="dash", line_color=te_colour)

fig.update_yaxes(title_text='SolO B [nT, GSM]', row=1, col=1)

for column, color in zip(['bx_scaled', 'by_scaled', 'bz_scaled', 'bt_scaled'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=df_scaled['time_shifted_exp'],
            y=df_scaled[column],
            name=column.upper(),
            line_color=color,
            showlegend=False
        ),
        row=2, col=1
    )
    
fig.add_trace(
    go.Scatter(
        x=df_scaled['time_shifted_exp'],
        y=df_scaled['bt_scaled_lb'],
        line_color='black',
        line_width=0,
        showlegend=False
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_scaled['time_shifted_exp'],
        y=df_scaled['bt_scaled_ub'],
        line_color='black',
        line_width=0,
        fill="tonextx",
        showlegend=False
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_scaled['time_shifted_exp'],
        y=df_scaled['bx_scaled_lb'],
        line_color='red',
        line_width=0,
        showlegend=False
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_scaled['time_shifted_exp'],
        y=df_scaled['bx_scaled_ub'],
        line_color='red',
        line_width=0,
        fill="tonextx",
        showlegend=False
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_scaled['time_shifted_exp'],
        y=df_scaled['by_scaled_lb'],
        line_color='green',
        line_width=0,
        showlegend=False
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_scaled['time_shifted_exp'],
        y=df_scaled['by_scaled_ub'],
        line_color='green',
        line_width=0,
        fill="tonextx",
        showlegend=False
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_scaled['time_shifted_exp'],
        y=df_scaled['bz_scaled_lb'],
        line_color='blue',
        line_width=0,
        showlegend=False
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_scaled['time_shifted_exp'],
        y=df_scaled['bz_scaled_ub'],
        line_color='blue',
        line_width=0,
        fill="tonextx",
        showlegend=False
    ),
    row=2, col=1
)

fig.add_vline(x=ts_s, row=2, col=1, line_width=1.5, line_dash="dash", line_color=shock_colour)
fig.add_vrect(x0=ts_s_lb, x1=ts_s_ub, row=2, col=1, line_width=0, fillcolor=shock_colour, opacity=0.3)
    
fig.add_vline(x=ts_le, row=2, col=1, line_width=1.5, line_dash="dash", line_color=le_colour)
fig.add_vrect(x0=ts_le_lb, x1=ts_le_ub, row=2, col=1, line_width=0, fillcolor=le_colour, opacity=0.3)

fig.add_vline(x=ts_te, row=2, col=1, line_width=1.5, line_dash="dash", line_color=te_colour)
fig.add_vrect(x0=ts_te_lb, x1=ts_te_ub, row=2, col=1, line_width=0, fillcolor=te_colour, opacity=0.3)
    
fig.update_yaxes(title_text='Pred SolO B [nT, GSM]', row=2, col=1)

for column, color in zip(['bx', 'by', 'bz', 'bt'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=dscovr_df['time'],
            y=dscovr_df[column],
            name=column.upper(),
            line_color=color,
            showlegend=False
        ),
        row=3, col=1
    )
    
fig.add_vrect(x0=ts_s_lb, x1=ts_s_ub, row=3, col=1, line_width=0, fillcolor=shock_colour, opacity=0.3)
fig.add_vrect(x0=ts_le_lb, x1=ts_le_ub, row=3, col=1, line_width=0, fillcolor=le_colour, opacity=0.3)
fig.add_vrect(x0=ts_te_lb, x1=ts_te_ub, row=3, col=1, line_width=0, fillcolor=te_colour, opacity=0.3)

fig.update_yaxes(title_text='DSCOVR B [nT, GSM]', row=3, col=1)

fig.add_trace(
    go.Scatter(
        x=dscovr_df['time'],
        y=dscovr_df['vt'],
        name='VT',
        line_color='purple',
        showlegend=True
    ),
    row=4, col=1
)
    
fig.add_vrect(x0=ts_s_lb, x1=ts_s_ub, row=4, col=1, line_width=0, fillcolor=shock_colour, opacity=0.3)
fig.add_vrect(x0=ts_le_lb, x1=ts_le_ub, row=4, col=1, line_width=0, fillcolor=le_colour, opacity=0.3)
fig.add_vrect(x0=ts_te_lb, x1=ts_te_ub, row=4, col=1, line_width=0, fillcolor=te_colour, opacity=0.3)

fig.update_yaxes(title_text='DSCOVR Vt [km/s]', row=4, col=1)

fig.add_trace(
    go.Scatter(
        x=dscovr_df['time'],
        y=dscovr_df['tp']/1000000,
        name='Tp',
        line_color='cornflowerblue',
        showlegend=True
    ),
    row=5, col=1, secondary_y=False,
)

fig.update_yaxes(title_text='DSCOVR Tp [MK]', row=5, col=1, secondary_y=False)

fig.add_trace(
    go.Scatter(
        x=dscovr_df['time'],
        y=dscovr_df['np'],
        name='Np',
        line_color='maroon',
        showlegend=True
    ),
    row=5, col=1, secondary_y=True,
)

fig.update_yaxes(title_text='DSCOVR Np [cm-3]', row=5, col=1, secondary_y=True)

fig.add_vrect(x0=ts_s_lb, x1=ts_s_ub, row=5, col=1, line_width=0, fillcolor=shock_colour, opacity=0.3)
fig.add_vrect(x0=ts_le_lb, x1=ts_le_ub, row=5, col=1, line_width=0, fillcolor=le_colour, opacity=0.3)
fig.add_vrect(x0=ts_te_lb, x1=ts_te_ub, row=5, col=1, line_width=0, fillcolor=te_colour, opacity=0.3)

fig.update_yaxes(showgrid=True, zeroline=False, showticklabels=True,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)
fig.update_xaxes(showgrid=True, zeroline=False, showticklabels=True, rangeslider_visible=False,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)

fig.update_layout(
    title=f'SolO LL MAG Data from {t_start} to {t_end} <br>Time-shifted using ELEvo predicted arrival time, B propto r^-1.6, and ICME duration propto r^0.8 <br>Latest SolO Position: r = {r_recent:.2f} AU, lat = {lat_recent:.2f}, lon = {lon_recent:.2f}, Time lag to current time: {lag:.2f} hours')

# fig.write_html(f'{output_path}/SolO_MAG_prediction_{t_now_date_hour}.html')

fig.show()